<a href="https://colab.research.google.com/github/czhurdlespeed/TFDevCert/blob/main/TFCert_Practice/08_introduction_to_nlp_in_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to NLP Fundamentals in TensorFlow

## Check for GPU

In [1]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-a686cf4b-5b49-6c52-507c-e3f8a76a9174)


## Import Helper Functions

In [2]:
!wget https://github.com/mrdbourke/tensorflow-deep-learning/raw/main/extras/helper_functions.py
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

--2024-02-18 16:38:16--  https://github.com/mrdbourke/tensorflow-deep-learning/raw/main/extras/helper_functions.py
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py [following]
--2024-02-18 16:38:16--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2024-02-18 16:38:16 (95.0 MB/s) - ‘helper_func

## Get a text dataset

- Using Kaggle's Introduction to NLP dataset

In [3]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
# Uzip data
unzip_data("nlp_getting_started.zip")

--2024-02-18 16:38:23--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.253.114.207, 172.253.119.207, 108.177.111.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.253.114.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.01s   

2024-02-18 16:38:23 (39.7 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



## Visualizing a text dataset

In [4]:
import pandas as pd
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
train_df["text"][100]

'.@NorwayMFA #Bahrain police had previously died in a road accident they were not killed by explosion https://t.co/gFJfgTodad'

In [6]:
# Shuffle training datafram
train_df_shuffled = train_df.sample(frac=1)
train_df_shuffled.head()

,id,keyword,location,text,target
2183,3130,debris,NaN,Debris found on Reunion Island comes from MH37...,1
626,902,bioterrorism,timeline kamu,Government Experts Concerned About Possible Bi...,1
2433,3495,derailed,NYC,Sad that biker beatdown derailed his pro-democ...,1
6024,8609,seismic,NaN,The 08/06/2015 AlabamaQuake seismic summary w/...,1
3747,5324,fire,NaN,Setting things on fire is always an option,1


In [7]:
# What does the test dataframe look like?
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [8]:
# How many examples of each class are there?
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [9]:
# How many locations?
train_df.location.value_counts()

USA                    104
New York                71
United States           50
London                  45
Canada                  29
                      ... 
MontrÌ©al, QuÌ©bec       1
Montreal                 1
ÌÏT: 6.4682,3.18287      1
Live4Heed??              1
Lincoln                  1
Name: location, Length: 3341, dtype: int64

In [10]:
# How many total samples?
len(train_df), len(test_df)

(7613, 3263)

In [11]:
# Let's visualize some random training examples
import random
random_index = random.randint(0, len(train_df)-5)
for row in train_df_shuffled[["text", "target"]][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f"Target: {target}", r"(real disaster)" if target > 0 else r"(not real disaster)")
  print(f"Text:\n{text}\n")
  print("---\n")

Target: 1 (real disaster)
Text:
Landslide in Italian Alps kills three: ROME (Reuters) - Three people were killed when a severe rainstorm in th... http://t.co/SmKZnF52Za

---

Target: 0 (not real disaster)
Text:
Do Your Own Thing: The Battle of Internal vs External Motivation: http://t.co/w9P3hAuHEi

---

Target: 1 (real disaster)
Text:
real magic in real life:

women went missing in Ohio town the focus of FBI probe after strange deaths and... http://t.co/6m0YNJWbc9

---

Target: 0 (not real disaster)
Text:
'I tried to save Mido Macia': One of the murder accused has testified he tried to save Mido MaciaÛªs life. http://t.co/vxVfAEEY0q

---

Target: 1 (real disaster)
Text:
Incredulous at continued outcry of welfare being a waste of taxpayers money but never similar objection to å£100bn on nuclear weapons

---



In [12]:
print("Hello\'\\ \n")

Hello'\ 



In [13]:
print(r"Hello\'\\ \n")

Hello\'\\ \n


### Split data into training and validation sets

In [14]:
from sklearn.model_selection import train_test_split
train_sentences, validation_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                                   train_df_shuffled["target"].to_numpy(),
                                                                                   test_size=0.1)


In [15]:
# Check the lengths
len(train_sentences), len(train_labels), len(validation_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [16]:
# Check the first 10 samples
train_sentences[:10], train_labels[:10]

(array(['Bigamist and his \x89Û÷first\x89Ûª wife are charged in the deaths of his \x89Û÷second\x89Ûª pregnant wife her child 8 her mother her nephew 1 and their u\x89Û_',
        '#hot  Funtenna: hijacking computers to send data as sound waves [Black Hat 2015] http://t.co/6AqrNanKFD #prebreak #best',
        'CLEARED:incident with injury:I-495  inner loop Exit 31 - MD 97/Georgia Ave Silver Spring',
        'Blending the old with the new in #Vanuatu to prepare for future emergencies: \nhttp://t.co/aFMKcFn1TL http://t.co/8QqzYZIAqf',
        '@MartinMJ22 @YouGov When did a 12 seat majority with 36% of the vote become a landslide?',
        'The Riddler would be the best early-exit primary presidential wannabe ever all certain of his chances until he gets wrecked by a rich guy',
        'IDEAS IN FOOD: Flattened http://t.co/QfrAWLn4BA',
        "Our tipster previews Chelsea v Swansea &amp; there's a 48/1 double! http://t.co/PFSrYJS1pc \n#Chelsea #Hazard http://t.co/SKdBot7TGF",
        'D

## Converting text into numbers

In [17]:
# Embedding - every word gets turned into a vector; you can specify the dimensions of the vector
  # gets updated as the network learns; therfore it can learn relationships between tokens
  # Pretrained embedding libraries e.g. word2vec, GloVe, etc.


In [18]:
# Text vectorization (tokenization)
train_sentences[:5]

array(['Bigamist and his \x89Û÷first\x89Ûª wife are charged in the deaths of his \x89Û÷second\x89Ûª pregnant wife her child 8 her mother her nephew 1 and their u\x89Û_',
       '#hot  Funtenna: hijacking computers to send data as sound waves [Black Hat 2015] http://t.co/6AqrNanKFD #prebreak #best',
       'CLEARED:incident with injury:I-495  inner loop Exit 31 - MD 97/Georgia Ave Silver Spring',
       'Blending the old with the new in #Vanuatu to prepare for future emergencies: \nhttp://t.co/aFMKcFn1TL http://t.co/8QqzYZIAqf',
       '@MartinMJ22 @YouGov When did a 12 seat majority with 36% of the vote become a landslide?'],
      dtype=object)

In [19]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
# use the default textvectorization parameters
text_vectorizer = TextVectorization(max_tokens=None, # how many words in our vocabulary; no cap on size of vocab
                                    standardize='lower_and_strip_punctuation',
                                    split='whitespace',
                                    ngrams=None, # create groups of n-words?
                                    output_mode='int', # how to map tokens to numbers
                                    output_sequence_length=None,
                                    pad_to_max_tokens=False)

In [20]:
# find the average number of tokens in the training tweets
round((sum([len(i.split()) for i in train_sentences]))/len(train_sentences))

15

In [21]:
# Setup text vectorization variables
max_vocab_length = 10000 # max num of words to have in vocab
max_length = 15 # max length our sequences will be

text_vectorizer = TextVectorization(max_tokens = max_vocab_length,
                                    output_mode='int',
                                    output_sequence_length=max_length)

In [22]:
# Fit the text vectorizer to the training text
text_vectorizer.adapt(train_sentences)

In [23]:
# Create a sample sentence and tokenize it
sample_sentence = "There's a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[297,   3, 216,   4,  13, 674,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [24]:
# Choose a random sentence from the training dataset and tokenize it
random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence} \
      \n\nVectorized versions:")
text_vectorizer([random_sentence])

Original text:
@pmharper don't worry I'm sure climate has nothing to do with hail floods tornados in Alberta &amp; Ontario. I'm sure god's just mad at you.       

Vectorized versions:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[4889,   59, 1154,   35,  482,  859,   43,  605,    5,   69,   14,
         330,  222, 4407,    4]])>

In [25]:
# Get the unique words in the vocab
words_in_vocab = text_vectorizer.get_vocabulary() # Get all the unique words in our training data
# Most common words
top_5_words = words_in_vocab[:5]
bottom_5_words = words_in_vocab[-5:]
print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"5 most common words: {top_5_words}")
print(f"5 least common words: {bottom_5_words}")

Number of words in vocab: 10000
5 most common words: ['', '[UNK]', 'the', 'a', 'in']
5 least common words: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


### Creating an Embedding using an Embedding Layer

In [26]:
# taken tokenized text into embedding vectors
from tensorflow.keras import layers
embedding = layers.Embedding(input_dim=max_vocab_length,
                             output_dim=128, # best to choose number divisible by 8
                             input_length=max_length # how long is each input
                             )
embedding

In [27]:
random_sentence = random.choice(train_sentences)
print(f"Original text:\n {random_sentence}\
\n\n Embedded version:")

sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
 INFO S. WND: 030/6. CLD: SCT014 BKN032. EXP INST APCH. RWY 05. CURFEW IN OPER UNTIL 2030 Z. TAXIWAYS FOXTROT 5 &amp; FOXTROT 6 NAVBL. TMP: 10.

 Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.01116318, -0.00023104,  0.01537604, ..., -0.0218329 ,
          0.0027502 , -0.01460532],
        [ 0.02733885,  0.04637471,  0.01936999, ..., -0.02557775,
          0.03232474,  0.04720524],
        [ 0.04237498, -0.02439121, -0.04461206, ...,  0.02241893,
          0.015316  , -0.01877921],
        ...,
        [-0.00928022,  0.03593396,  0.0195469 , ..., -0.01726277,
          0.0217347 ,  0.03351785],
        [ 0.02228017, -0.00146062,  0.01896827, ..., -0.0174969 ,
         -0.02417191,  0.04502961],
        [ 0.01481039, -0.03510704, -0.04824306, ..., -0.0473081 ,
          0.0267378 , -0.0407159 ]]], dtype=float32)>

In [28]:
# Check out a single token's embedding
sample_embed[0][0], sample_embed[0][0].shape, random_sentence[0]

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([ 0.01116318, -0.00023104,  0.01537604, -0.04380932,  0.00046848,
        -0.02065368, -0.028667  ,  0.00728363, -0.00058621, -0.04094565,
         0.04376309, -0.04982552, -0.02970414,  0.03377532,  0.00172266,
         0.02922808,  0.02247194,  0.01594981, -0.02912582, -0.03611034,
         0.01974508, -0.02486029, -0.04361843, -0.04173392,  0.0289104 ,
         0.03523368, -0.02796942, -0.03956926,  0.02595807, -0.03076473,
        -0.0332006 , -0.01725693, -0.04114943,  0.03444839, -0.0454425 ,
         0.02407907, -0.01055314,  0.01727364,  0.02594874, -0.01201764,
         0.01128031,  0.01663199, -0.00880162, -0.03592789,  0.03385257,
        -0.00407522, -0.03362022, -0.04196379, -0.03539545,  0.00974949,
         0.02711363,  0.04502882,  0.03167329,  0.04115876,  0.01433947,
        -0.01106248, -0.0383944 , -0.00485926,  0.03237299,  0.028936  ,
        -0.035709  ,  0.03581489, -0.04834793,  0.01266147, -0.02845281,
  

## Modeling a text dataset
* Model 0: Naive Bayes (baseline) [sklearn ML map](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html)
* Model 2: LSTM model (RNN)
* Model 3: GRU model (RNN)
* Model 4: Bidirectional-LSTM model (RNN)
* Model 5: 1D Convolutional NN (CNN)
* Model 6: TF Hub Pretrained Feature Extractor (transfer learning)
* Model 7: Same as model 6 with 10% of training data

### Model 0: Getting a baseline

In [29]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer # turn text into numbers/vectors
from sklearn.pipeline import Pipeline
model_0 = Pipeline([
    ("tfidf", TfidfVectorizer()),
    ("clf", MultinomialNB()) # clf == "classifier"
])
model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [30]:
# Evaluate our baseline model
baseline_score = model_0.score(validation_sentences, val_labels)
print(f"Our baseline model achieves an accuracy of: {baseline_score*100:.2f}%")

Our baseline model achieves an accuracy of: 82.55%


In [31]:
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [32]:
# Make predictions
baseline_preds = model_0.predict(validation_sentences)
baseline_preds[:20]

array([0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0])

### Creating an evaluation function for our model experiments

Create on to compare our model's predictions with the truth labels
* Accuracy
* Precision
* Recall
* F1-score

In [33]:
# Function to evaluate: accuracy, precision, recall, F1-score
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
def calculate_results(y_true, y_pred):
  """
  Calculates model accuracy, precision, recall and f1 score of a binary classification model
  """
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  # Calculate model precision, recall, and f1-score using "weighted" average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                   "precision": model_precision,
                   "recall": model_recall,
                   "f1": model_f1}
  return model_results

In [34]:
baseline_results = calculate_results(y_true=val_labels,
                                     y_pred = baseline_preds)
baseline_results

{'accuracy': 82.54593175853019,
 'precision': 0.8343718354103375,
 'recall': 0.8254593175853019,
 'f1': 0.8211880892810138}

# Model 1: Feed-forward neural network (dense model)

In [35]:
# Create a tensorboard callback (need to create a new one for each model)
from helper_functions import create_tensorboard_callback

# Create a directory to save Tensorboard logs
SAVE_DIR = "model_logs"

# Build model with the Functional API
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype=tf.string) # inputs are 1D strings
x = text_vectorizer(inputs) # turn input text into numbers
x = embedding(x) # create an embedding of the numberized inputs
x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1, activation="sigmoid")(x) # Create output layer, want binary outputs
model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense")

In [36]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (  (None, 128)               0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1280129 (4.88 MB)
Trainable params: 128

In [37]:
# Compile model
model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [38]:
# Fit model
model_1_history = model_1.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(validation_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                     experiment_name="model_1_dense")])

Saving TensorBoard log files to: model_logs/model_1_dense/20240218-163832
Epoch 1/5
215/215 [==============================] - 11s 35ms/step - loss: 0.6116 - accuracy: 0.6941 - val_loss: 0.5093 - val_accuracy: 0.7979
Epoch 2/5
215/215 [==============================] - 2s 10ms/step - loss: 0.4435 - accuracy: 0.8171 - val_loss: 0.4350 - val_accuracy: 0.8176
Epoch 3/5
215/215 [==============================] - 1s 6ms/step - loss: 0.3488 - accuracy: 0.8603 - val_loss: 0.4268 - val_accuracy: 0.8150
Epoch 4/5
215/215 [==============================] - 1s 6ms/step - loss: 0.2856 - accuracy: 0.8869 - val_loss: 0.4267 - val_accuracy: 0.8163
Epoch 5/5
215/215 [==============================] - 1s 6ms/step - loss: 0.2381 - accuracy: 0.9101 - val_loss: 0.4429 - val_accuracy: 0.8123


In [39]:
# Check the results
model_1.evaluate(validation_sentences, val_labels)

24/24 [==============================] - 0s 3ms/step - loss: 0.4429 - accuracy: 0.8123


[0.44285649061203003, 0.8123359680175781]

In [40]:
# make some predictions and evaluate those
model_1_pred_probs = model_1.predict(validation_sentences)
model_1_pred_probs

24/24 [==============================] - 0s 2ms/step


array([[0.16665865],
       [0.08394734],
       [0.9933682 ],
       [0.0096827 ],
       [0.04736165],
       [0.37891915],
       [0.9757362 ],
       [0.15932323],
       [0.08831844],
       [0.35273802],
       [0.00434855],
       [0.1016547 ],
       [0.04112943],
       [0.09963124],
       [0.99925023],
       [0.7490465 ],
       [0.93360364],
       [0.13815342],
       [0.01316588],
       [0.24223742],
       [0.99926656],
       [0.91958654],
       [0.1499649 ],
       [0.5371941 ],
       [0.00615489],
       [0.9431859 ],
       [0.08523015],
       [0.12563635],
       [0.96548724],
       [0.9758961 ],
       [0.03486154],
       [0.01901238],
       [0.66341925],
       [0.97458875],
       [0.2144119 ],
       [0.9980653 ],
       [0.19931135],
       [0.8183929 ],
       [0.3202855 ],
       [0.54042065],
       [0.55273217],
       [0.9902137 ],
       [0.02293248],
       [0.00851741],
       [0.03769625],
       [0.9900153 ],
       [0.35389975],
       [0.008

In [41]:
# Convert model prediction probabilites to label format
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))
model_1_preds

<tf.Tensor: shape=(762,), dtype=float32, numpy=
array([0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1.,
       0., 0., 0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 1., 0., 0., 1., 1.,
       0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 1., 0., 0., 1., 0., 0.,
       1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 1., 1.,
       1., 0., 1., 0., 1., 0., 0., 1., 0., 1., 0., 1., 0., 1., 0., 0., 0.,
       0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0.,
       1., 1., 1., 0., 0., 0., 1., 0., 1., 1., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1., 1., 1., 0., 1., 0., 1., 1., 0., 1., 1., 1., 0., 0., 1.,
       0., 1., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 0., 0., 0., 1., 1.,
       1., 0., 1., 0., 0., 1., 0., 1., 1., 1., 0., 1., 0., 1., 0., 1., 0.,
       0., 1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       1., 1., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 1.,
       1., 1., 0., 1., 1., 0., 0., 1., 0., 0., 0., 1

In [42]:
# Calculate our model_1 results
model_1_results = calculate_results(y_true=val_labels,
                                    y_pred=model_1_preds)
model_1_results

{'accuracy': 81.23359580052494,
 'precision': 0.811817411097949,
 'recall': 0.8123359580052494,
 'f1': 0.81156043327798}

In [43]:
baseline_results

{'accuracy': 82.54593175853019,
 'precision': 0.8343718354103375,
 'recall': 0.8254593175853019,
 'f1': 0.8211880892810138}

In [44]:
import numpy as np
np.array(list(model_1_results.values())) > np.array(list(baseline_results.values()))

array([False, False, False, False])

## Visualizing learned embeddings

In [45]:
# Get the vocabulary from the text vectorization layer
words_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab), words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [46]:
# Model 1 Summary
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (  (None, 128)               0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1280129 (4.88 MB)
Trainable params: 128

In [54]:
# Get the weight matrix of embedding layer
# (numerical representations of each token in our training data; learned for 5 epochs)
embed_weights = model_1.get_layer("embedding").get_weights()[0]
embed_weights

array([[-0.03556719, -0.02439384,  0.00533569, ..., -0.04615637,
         0.0262187 ,  0.02303373],
       [-0.04027543, -0.03978007, -0.01868589, ..., -0.04876405,
         0.05134028, -0.01287759],
       [-0.03259987,  0.02813799, -0.01368915, ..., -0.00493422,
        -0.01037545,  0.01860335],
       ...,
       [ 0.01865206, -0.00266247,  0.01152065, ...,  0.03219048,
         0.02502009,  0.03493685],
       [-0.09525589, -0.00581831, -0.04408037, ..., -0.07592053,
         0.02619899,  0.01186154],
       [-0.08312871, -0.0546471 , -0.06789241, ..., -0.01786359,
         0.08635088,  0.01649686]], dtype=float32)

In [56]:
print(embed_weights.shape) # same size as vocab size and embedding_dim; each word embedded into 128 dim vector

(10000, 128)


# [Tensorflow Embedding Visualization Tool](https://projector.tensorflow.org/)

In [57]:
# Create embedding files
import io
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(words_in_vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = embed_weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

In [58]:
# Download files from Colab to upload to Tensorflow projector (see link above)
try:
  from google.colab import files
  files.download('vectors.tsv')
  files.download('metadata.tsv')
except Exception:
  pass

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Recurrent Neural Networks (RNN's)
* Sequence data
* Use representation of previous input to aid the representation of a later input

### Model 2: LSTM
LSTM = long short term memory (one of th most popular Recurrent Cells)

RNN Structure:

```
Input (text) -> Tokenize -> Embedding -> Layers (RNNs/dense) -> Output (label probability)
```

In [63]:
# Create an LSTM Model
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs) # map text to numbers/tokens
x = embedding(x) # create embedding vectors from tokens
#print(x.shape)
#x = layers.LSTM(units=64, return_sequences=True)(x) # returns a squence; maintains dims of a sequence
# when you're stacking RNN cells together, you need to set return_sequences=True
#   i.e. input to LSTM expected to be a sequence; the output doesn't necessarily have to be a sequence
#print(x.shape)
x = layers.LSTM(64)(x)
#print(x.shape)
#x = layers.Dense(64, activation='relu')(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_2 = tf.keras.Model(inputs,outputs, name="model_2_LSTM")

In [64]:
model_2.summary()

Model: "model_2_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 lstm_5 (LSTM)               (None, 64)                49408     
                                                                 
 dense_5 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1329473 (5.07 MB)
Trainable params: 1329473 (5.07 MB)
Non-trainable params: 0 (0.00 Byte)
________________

In [65]:
# Compile the model
model_2.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [67]:
# Fit the model
model_2_history = model_2.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(validation_sentences,val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                     "model_2_LSTM")])

Saving TensorBoard log files to: model_logs/model_2_LSTM/20240218-172349
Epoch 1/5
215/215 [==============================] - 19s 68ms/step - loss: 0.2154 - accuracy: 0.9216 - val_loss: 0.5589 - val_accuracy: 0.8045
Epoch 2/5
215/215 [==============================] - 4s 19ms/step - loss: 0.1540 - accuracy: 0.9432 - val_loss: 0.5780 - val_accuracy: 0.8018
Epoch 3/5
215/215 [==============================] - 4s 17ms/step - loss: 0.1230 - accuracy: 0.9552 - val_loss: 0.7306 - val_accuracy: 0.7966
Epoch 4/5
215/215 [==============================] - 3s 13ms/step - loss: 0.1011 - accuracy: 0.9625 - val_loss: 0.7581 - val_accuracy: 0.7992
Epoch 5/5
215/215 [==============================] - 2s 11ms/step - loss: 0.0767 - accuracy: 0.9695 - val_loss: 0.7233 - val_accuracy: 0.8045


In [70]:
# Make predictions with LSTM model
model_2_pred_probs = model_2.predict(validation_sentences)
model_2_pred_probs[:10]

24/24 [==============================] - 1s 4ms/step


array([[0.01720196],
       [0.04032976],
       [0.9998903 ],
       [0.00184612],
       [0.00690784],
       [0.44833583],
       [0.99965644],
       [0.14691654],
       [0.00365316],
       [0.39848474]], dtype=float32)

In [72]:
# convert model 2 pred probs to labels
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 0., 1., 0., 0., 0., 1., 0., 0., 0.], dtype=float32)>

In [73]:
# Calculate model 2 results
model_2_results = calculate_results(val_labels, model_2_preds)
model_2_results

{'accuracy': 80.4461942257218,
 'precision': 0.80385748255827,
 'recall': 0.8044619422572179,
 'f1': 0.8037513068281177}

In [75]:
# But baseline is still better
baseline_results

{'accuracy': 82.54593175853019,
 'precision': 0.8343718354103375,
 'recall': 0.8254593175853019,
 'f1': 0.8211880892810138}

### Model 3: RNN with GRU Cell